In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# codigo generar csv con las predicciones de los paises

# cargamos las librerias a utilizar
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# hacemos la ingesta de la data de remesas por paises
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MLparaPFbootcampDS/dwhMLVULprepDATA.csv')

# Seleccionar país para la predicción
paises= ['PER', 'COL','BOL', 'MEX','CHL', 'ECU','BRA', 'URY', 'VEN']
indice= 1
paispredecir = paises[indice]
df_ml = df.copy()
df_ml['prediccion']= 0

# Convertir de ancho a largo
df_melted = pd.melt(df, id_vars=['codigopais'], var_name='year', value_name='remittances')

# eliminamos valores NaN
df_melted.dropna(inplace=True)

# Convertir year a variable numérica
df_melted['year'] = pd.to_numeric(df_melted['year'])

# Convertir remittances a tipo float
#df_melted['remittances'] = df_melted['remittances'].str.replace(',', '.').astype(float)

# Convertir codigopais a variable numérica mediante LabelEncoder
le = LabelEncoder()
df_melted['codigopais'] = le.fit_transform(df_melted['codigopais'])


for indice in range(len(paises)):
  paispredecir = paises[indice]
 
  # falta añadir una consistencias que los paises seleccionados
  # tengan existan en la tabla con datos, sino deberia seguir con el siguiente pais...
  
  # Filtrar datos por país seleccionado
  df_pais = df_melted[df_melted['codigopais'] == le.transform([paispredecir])[0]]

  # Ajustar modelo de regresión
  X = df_pais[df_pais['year'] <= 2020][['year', 'codigopais']]
  y = df_pais[df_pais['year'] <= 2020]['remittances']
  reg = LinearRegression().fit(X, y)

  # Hacer predicciones para el país seleccionado
  añopredecir = 2021
  X_pred = np.array([[añopredecir, le.transform([paispredecir])[0]]])
  y_pred = reg.predict(X_pred)
  print(f"Predicción para {paispredecir} en {añopredecir}: {y_pred[0]:.4f}")

  df_ml.loc[df_ml['codigopais'] == paispredecir,'prediccion'] = round(y_pred[0], 4)


# al finalizar el for, guarda el nuevo .csv del dataframe: df_ml
df_ml.to_csv('/content/drive/MyDrive/Colab Notebooks/MLparaPFbootcampDS/dwhMLVULprepDATAPREDICCIONML.csv', index = False)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature n

Predicción para PER en 2021: -2.6788
Predicción para COL en 2021: 932.8856
Predicción para BOL en 2021: 4.3838
Predicción para MEX en 2021: 39.5127
Predicción para CHL en 2021: -4.0605
Predicción para ECU en 2021: 18.9817
Predicción para BRA en 2021: 23.8985
Predicción para URY en 2021: 0.8097
Predicción para VEN en 2021: 168740.8758
